In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from utils import (
    DEGREES_PER_PIXEL,
    TIMESTAMP_IDENT,
    X_PIXELS,
    Y_PIXELS,
    Events,
    extract_gaze_data_between_timestamps_proper,
    get_participant_dominant_eye,
    load_participants_metadata,
)
import json

from velocityThreshold import detect_fix_ivt, find_sacc_from_fix

EYE_TRACKER_FOLDER = "eye_tracker_data/"
GAZE_DATA = []
GAZE_DATA_BOOK = []
GAZE_DATA_PAGE = []

# participants that were told that they can change the settings beforehand
participant_metadata =  load_participants_metadata()
participant_ids = [int(participant) for participant, data in participant_metadata.items() if data["split"] == 2 and data["low_resolution"] == False]

subquery = Events.select().where(Events.participant_id.in_(participant_ids))


In [ ]:
import matplotlib.pyplot as plt
import math

import pandas as pd
from utils import DEGREES_PER_PIXEL, TIMESTAMP_IDENT, X_PIXELS, Y_PIXELS, get_participant_dominant_eye, identify_saccade_type_with_color
from velocityThreshold import detect_fix_ivt


def plot_fixations_on_screenshot(participant_id, fixations, saccades, screenshot_path, title, saccadic_threshold):
    fig, ax = plt.subplots(figsize=(X_PIXELS / 100, Y_PIXELS / 100))
    # set ax limits
    ax.set_xlim(0, X_PIXELS)
    ax.set_ylim(Y_PIXELS, 0)
    img = plt.imread(screenshot_path)

    p = ax.scatter(fixations["x"], fixations["y"], c=fixations["ts"], s=40, cmap="plasma")

    # take successive fixations and draw an arrow between them.
    for a, b in zip(fixations.index[:-1], fixations.index[1:]):
        point_a = (fixations["x"][a], fixations["y"][a])
        point_b = (fixations["x"][b], fixations["y"][b])
        # draw an arrow between the two points

        color = identify_saccade_type_with_color(point_a, point_b)

        ax.arrow(
            point_a[0],
            point_a[1],
            point_b[0] - point_a[0],
            point_b[1] - point_a[1],
            color=color,
            alpha=0.5,
            width=0.1,
            head_width=10,
            length_includes_head=True,
        )
    
    # add a key for the arrow colors
    ax.scatter([], [], c="green", label="Saccade")
    ax.scatter([], [], c="blue", label="Return Sweep")
    ax.scatter([], [], c="red", label="Regression")
    ax.scatter([], [], c="purple", label="Other")
    ax.legend()

    # show color bar for timestamps
    cbar = fig.colorbar(p, ax=ax)
    cbar.set_label("Timestamp, with darker colors being earlier in the trial")

    ax.imshow(img, extent=[0, X_PIXELS, Y_PIXELS, 0])
    ax.set_title(title)


In [ ]:
for participant in participant_ids:
    book_open_events = subquery.where(
        (Events.event == "OPEN_BOOK")
        & (Events.new_value == "Chasing Sunsets - ")
        & (Events.participant_id == participant)
    )

    for book in book_open_events:
        START_TIME_BOOK = book.time
        book_end = (
            subquery.where(Events.event == "CLOSE_BOOK")
            .where(Events.time > START_TIME_BOOK)
            .get()
        )
        END_TIME_BOOK = book_end.time

        # Get the events that have a timestamp less than the first CLOSE_BOOK event
        events_book = subquery.where(Events.time <= END_TIME_BOOK)

        START_TIME_BOOK /= 1000
        formatted_time = datetime.fromtimestamp(START_TIME_BOOK).strftime(
            "%Y-%m-%d_%H-%M-%S"
        )

        GAZE_FILE = f"{EYE_TRACKER_FOLDER}[{participant}]-{formatted_time}.json"
        with open(GAZE_FILE, "r") as f:
          GAZE_DATA_BOOK = json.load(f)
        
        DOMINANT_EYE = get_participant_dominant_eye(participant)

        # get per-page gaze data
        pages = subquery.where(
            (Events.event.contains("_PAGE") | (Events.event == "OPEN_BOOK"))
            & (Events.time >= START_TIME_BOOK)
            & (Events.time < END_TIME_BOOK)
        )
        for page in pages:
            START_TIME_PAGE = page.time
            page_end = subquery.where(
                (Events.event.contains("_PAGE")) & (Events.time > START_TIME_PAGE)
            ).get_or_none()
            if page_end is None:
                continue
            END_TIME_PAGE = page_end.time

            GAZE_DATA_PAGE = extract_gaze_data_between_timestamps_proper(
                GAZE_DATA_BOOK, START_TIME_PAGE, END_TIME_PAGE
            )

            timestamps = []
            x = []
            y = []
            # for each packet, plot the gaze point
            for packet in GAZE_DATA_PAGE["data"]:
                if packet[f"{DOMINANT_EYE}_gaze_point_validity"] == 0:
                    continue
                if not (0 <= packet[f"{DOMINANT_EYE}_gaze_point_on_display_area"][0] <= 1) or not ( 0 <= packet[f"{DOMINANT_EYE}_gaze_point_on_display_area"][1] <= 1):
                    continue
                x.append(
                    (packet[f"{DOMINANT_EYE}_gaze_point_on_display_area"][0] * X_PIXELS)
                )
                y.append(
                    (packet[f"{DOMINANT_EYE}_gaze_point_on_display_area"][1] * Y_PIXELS)
                )
                timestamps.append(packet[TIMESTAMP_IDENT])

            df = pd.DataFrame({"x": x, "y": y, "ts": timestamps})
            df = df.sort_values(by="ts")
            df = df.reset_index(drop=True)

            df["x"] = df["x"] * DEGREES_PER_PIXEL
            df["y"] = df["y"] * DEGREES_PER_PIXEL
            df["ts"] = df["ts"] / 1_000_000

            if df.empty:
                continue

            # display(df)
            SACCADIC_THRESHOLD = 100
            # Plot fixations
            fixations, v, labels = detect_fix_ivt(df, sacvel=SACCADIC_THRESHOLD)
            if fixations.empty:
                continue
            saccades_for_page = find_sacc_from_fix(fixations)

            fixations["x"] = fixations["x"] / DEGREES_PER_PIXEL
            fixations["y"] = fixations["y"] / DEGREES_PER_PIXEL
            screenshot_path = page.screenshot_file
            plot_fixations_on_screenshot(participant, fixations, saccades_for_page, screenshot_path, title=f"Participant {participant}, [{page.id}] {page.event}, avg. saccadic length: {np.mean(saccades_for_page['dxy'])}", saccadic_threshold=SACCADIC_THRESHOLD)
        break


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Wedge

fig, ax = plt.subplots(figsize=(5, 5))
ax.set_xlim(0, 10)
ax.set_ylim(0, 10)
ax.legend()

# Draw a filled circle
circle = plt.Circle((5, 5), 5, color='black', fill=True)
ax.add_patch(circle)

# Define the colors, labels, and angles
colors = ["green", "purple", "red", "blue", "purple",  "green"]
labels = ["Saccade", "Return Sweep", "Regression", "Other"]
angles = [0, 20, 45, 182, 200, 340, 360]

# Plot the angles and fill them in with the appropriate color
for i in range(len(angles)-1):
    angle_start = angles[i]
    angle_end = angles[i+1]

    theta1 = angle_start
    theta2 = angle_end
    center = (5, 5)
    radius = 5

    wedge = Wedge(center=center, r=radius, theta1=theta1, theta2=theta2, facecolor=colors[i])
    ax.add_patch(wedge)

# add legend
ax.scatter([], [], c="green", label="Saccade")
ax.scatter([], [], c="blue", label="Return Sweep")
ax.scatter([], [], c="red", label="Regression")
ax.scatter([], [], c="purple", label="Other")

# make legend outisde of plot
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# add title
ax.set_title("Key for Saccade Types")

# hide axis
ax.axis('off')

# write angle labels
ax.text(5, 0.5, "270°", ha="center")
ax.text(9.5, 5, "0°", va="center")
ax.text(5, 9.5, "90°", ha="center")
ax.text(0.5, 5, "180°", va="center")


plt.show()